## Path finding algorithms

There are descriptions and implementations of differnet path finding algortihms. Because of failure with installing and using standard **ROS** libraries (like chomp, sbpl and odom) I decided to implement them by myself :)

### 1.  RRT algorithm 
(Here we assume that the map of the given environtment is unknown)

The path is planned by building a tree starting from the position of the robot. When a point in the space is randomly sampled, it is checked if that point collides with an obstacle in the space. If the sampled point has no collisions, it is then checked if the straight line path between the sampled point and the nearest existing point in the tree has any collisions. If this straight line path has no collisions, the sampled point is added to the tree with the nearest point as its parent node. If there is a collision, this point is thrown out.

<img src="sample_result.jpg">

In [29]:
import sys, random, math, pygame
from pygame.locals import *
from math import sqrt, cos, sin, atan2

class Node(object):
    """Node in a tree"""
    def __init__(self, point, parent):
        super(Node, self).__init__()
        self.point = point
        self.parent = parent


def dist(p1,p2):
    return sqrt((p1[0]-p2[0])*(p1[0]-p2[0])+(p1[1]-p2[1])*(p1[1]-p2[1]))

def point_circle_collision(p1, p2, radius):
    distance = dist(p1,p2)
    if (distance <= radius):
        return True
    return False



XDIM = 720
YDIM = 500
WINSIZE = [XDIM, YDIM]
EPSILON = 7.0
NUMNODES = 5000
GOAL_RADIUS = 10
MIN_DISTANCE_TO_ADD = 1.0
GAME_LEVEL = 1

pygame.init()
fpsClock = pygame.time.Clock()

screen = pygame.display.set_mode(WINSIZE)
pygame.display.set_caption('Rapidly Exploring Random Tree')
white = 255, 240, 200
black = 20, 20, 40
red = 255, 0, 0
blue = 0, 255, 0
green = 0, 0, 255
cyan = 0,255,255


count = 0
rectObs = []

def step_from_to(p1,p2):
    if dist(p1,p2) < EPSILON:
        return p2
    else:
        theta = atan2(p2[1]-p1[1],p2[0]-p1[0])
        return p1[0] + EPSILON*cos(theta), p1[1] + EPSILON*sin(theta)

def collides(p):
    for rect in rectObs:
        if rect.collidepoint(p) == True:
            return True
    return False


def get_random():
    return random.random()*XDIM, random.random()*YDIM

def get_random_clear():
    while True:
        p = get_random()
        noCollision = collides(p)
        if noCollision == False:
            return p


def init_obstacles(configNum):
    global rectObs
    rectObs = []
    print("config "+ str(configNum))
    if (configNum == 0):
        rectObs.append(pygame.Rect((XDIM / 2.0 - 50, YDIM / 2.0 - 100),(100,200)))
    if (configNum == 1):
        rectObs.append(pygame.Rect((40,10),(100,200)))
        rectObs.append(pygame.Rect((500,200),(500,200)))
    if (configNum == 2):
        rectObs.append(pygame.Rect((40,10),(100,200)))
    if (configNum == 3):
        rectObs.append(pygame.Rect((40,10),(100,200)))

    for rect in rectObs:
        pygame.draw.rect(screen, red, rect)


def reset():
    global count
    screen.fill(black)
    init_obstacles(GAME_LEVEL)
    count = 0


def main():
    global count

    initPoseSet = False
    initialPoint = Node(None, None)
    goalPoseSet = False
    goalPoint = Node(None, None)
    currentState = 'init'

    nodes = []
    reset()

    while True:
        if currentState == 'init':
            print('set the goal point')
            fpsClock.tick(10)
        elif currentState == 'goalFound':
            #traceback
            currNode = goalNode.parent
            while currNode.parent != None:
                pygame.draw.line(screen,cyan,currNode.point,currNode.parent.point)
                currNode = currNode.parent
            optimizePhase = True
        elif currentState == 'optimize':
            fpsClock.tick(0.5)
            pass
        elif currentState == 'buildTree':
            count = count+1
            if count < NUMNODES:
                foundNext = False
                while foundNext == False:
                    rand = get_random_clear()
                    parentNode = nodes[0]

                    for p in nodes: 
                        if dist(p.point,rand) <= dist(parentNode.point,rand): 
                            newPoint = step_from_to(p.point,rand)
                            if collides(newPoint) == False: 
                                parentNode = p 
                                foundNext = True

                newnode = step_from_to(parentNode.point,rand)
                nodes.append(Node(newnode, parentNode))
                pygame.draw.line(screen,white,parentNode.point,newnode)

                if point_circle_collision(newnode, goalPoint.point, GOAL_RADIUS):
                    currentState = 'goalFound'
                    goalNode = nodes[len(nodes)-1]

                if count%100 == 0:
                    print("node: " + str(count))
            else:
                print("Ran out of nodes... :(")
                return;

    
        for e in pygame.event.get():
            if e.type == QUIT or (e.type == KEYUP and e.key == K_ESCAPE):
                sys.exit("Exiting")
            if e.type == MOUSEBUTTONDOWN:
                print('mouse down')
                if currentState == 'init':
                    if initPoseSet == False:
                        nodes = []
                        if collides(e.pos) == False:
                            print('initiale pose set: '+str(e.pos))

                            initialPoint = Node(e.pos, None)
                            nodes.append(initialPoint) 
                            initPoseSet = True
                            pygame.draw.circle(screen, blue, initialPoint.point, GOAL_RADIUS)
                    elif goalPoseSet == False:
                        print('goal pose set: '+str(e.pos))
                        if collides(e.pos) == False:
                            goalPoint = Node(e.pos,None)
                            goalPoseSet = True
                            pygame.draw.circle(screen, green, goalPoint.point, GOAL_RADIUS)
                            currentState = 'buildTree'
                else:
                    currentState = 'init'
                    initPoseSet = False
                    goalPoseSet = False
                    reset()

        pygame.display.update()
        fpsClock.tick(10000)

if __name__ == '__main__':
    main()
input("press Enter to quit")

config 1
set the goal point
set the goal point
set the goal point
set the goal point
set the goal point
set the goal point
set the goal point
set the goal point
mouse down
initiale pose set: (143, 479)
set the goal point
set the goal point
set the goal point
set the goal point
set the goal point
mouse down
goal pose set: (347, 215)
node: 100


SystemExit: Exiting

To exit: use 'exit', 'quit', or Ctrl-D.


## Now lets assume that we know the map:

<img src="canvas.png">
Here, **s**-is the start position, **g** -is the goal and the grey cells express the obstacles. Now lets test different path planning algorithms for the given map(as far as I know on the competition the same environment will be given).

### 2. Depth first algorithm

 In depth-first search the idea is to travel as deep as possible from neighbour to neighbour before backtracking. What determines how deep is possible is that you must follow edges, and you don't visit any vertex twice.

To do this properly we need to keep track of which vertices have already been visited, plus how we got to (the path to...) where we currently are, so that we can backtrack.

For example, for the following graph,
<img src="200px-Graph.traversal.example.svg.png">
a depth-first search starting at A, assuming that the left edges in the shown graph are chosen before right edges, and assuming the search remembers previously visited nodes and will not repeat them (since this is a small graph), will visit the nodes in the following order: A, B, D, F, E, C, G. The edges traversed in this search form a Trémaux tree, a structure with important applications in graph theory.

Performing the same search without remembering previously visited nodes results in visiting nodes in the order A, B, D, F, E, A, B, D, F, E, etc. forever, caught in the A, B, D, F, E cycle and never reaching C or G.


In [34]:
class SearchGrid:
    def build(self):
        self.grid =np.arange(64).reshape(8,8) + 1    
        self.start_pos = [5,3]                      
        self.goal_pos = [2,2]                       
        for i in range(8):
            for j in range(8):
                temp = self.grid[i,j]
                if temp >= 27 and temp <= 31 or temp == 34 or \
                    temp == 43 or temp == 47:
                    self.grid[i,j] = -1

    def position(self, value):
         return (np.nonzero(self.grid == value)[0][0], \
            np.nonzero(self.grid == value)[1][0])
    
    def heuristic(self, value):
        position = self.position(value)             
        return np.abs(self.goal_pos[0] - position[0]) + np.abs(self.goal_pos[1] - position[1])



class Search:

    def __init__(self, search_grid):
        self.search_grid = search_grid    
        self.node_parent_list = []        
        self.fringe = []                  
        self.visited = []                 
                                          
        self.expanded = []              
        self.path = []                    
  
    def set_path(self, goal):
        current = goal;
        self.path.insert(0, current);     

        for tup in reversed(self.node_parent_list):
            if tup[0] == current:
                current = tup[1]              
                self.path.insert(0, current)

class DepthFirst(Search):
    def search(self):
       
        start = self.search_grid.grid[self.search_grid.start_pos[0], \
            self.search_grid.start_pos[1]]
        goal = self.search_grid.grid[self.search_grid.goal_pos[0], \
            self.search_grid.goal_pos[1]]

        self.fringe.append(start)         

        while True:
            if len(self.fringe) == 0:
                return False
            current = self.fringe.pop(0)
            
            if self.visited.count(current) == 0:
                self.expanded.append(current) 
                self.visited.append(current)  
            else:
                continue                
        
            if current == goal:
                self.set_path(goal)
                return True
                
            self.__expand(current)          
        
    def __expand(self, current):
        

        pos = self.search_grid.position(current)
        current_fringe = []
        if pos[0] > 0:
            current_fringe.insert(0, self.search_grid.grid[pos[0] - 1, pos[1]])
        if pos[1] > 0:
            current_fringe.insert(0, self.search_grid.grid[pos[0], pos[1] - 1])
        if pos[1] < (self.search_grid.grid.shape[1] - 1):
            current_fringe.insert(0, self.search_grid.grid[pos[0], pos[1] + 1])
        if pos[0] < (self.search_grid.grid.shape[0] - 1):
            current_fringe.insert(0, self.search_grid.grid[pos[0] + 1, pos[1]])
        for value in current_fringe:
            if value != -1:
                self.fringe.insert(0, value)
                self.node_parent_list.append((value, current))  

In [35]:
import numpy as np

grid = SearchGrid()
grid.build()

print "mesh has been built!"
sol = DepthFirst(grid)
sol.search()
print "\nExpanded: ", sol.expanded
print "Path: ", sol.path
print "Path Cost: ",


if (len(sol.path) > 0):
    print len(sol.path) - 1
else:
    print "Failure!"



mesh has been built!

Expanded:  [44, 36, 35, 37, 38, 39, 40, 32, 24, 16, 8, 7, 6, 5, 4, 3, 2, 1, 9, 10, 11, 12, 13, 14, 15, 23, 22, 21, 20, 19]
Path:  [44, 36, 37, 38, 39, 40, 32, 24, 16, 8, 7, 6, 5, 4, 3, 2, 1, 9, 10, 11, 12, 13, 14, 15, 23, 22, 21, 20, 19]
Path Cost:  28


### 3. Breadth first algorithm

Breadth-first search is a way to find all the vertices reachable from the a given source vertex, s. Like depth first search, BFS traverse a connected component of a given graph and defines a spanning tree. Intuitively, the basic idea of the breath-first search is this: send a wave out from source s. The wave hits all vertices 1 edge from s. From there, the wave hits all vertices 2 edges from s. Etc. We use FIFO queue Q to maintain the wavefront: v is in Q if and only if wave has hit v but has not come out of v yet. 

Breadth-first search starts at a given vertex s, which is at level 0. In the first stage, we visit all the vertices that are at the distance of one edge away. When we visit there, we paint as "visited," the vertices adjacent to the start vertex s - these vertices are placed into level 1. In the second stage, we visit all the new vertices we can reach at the distance of two edges away from the source vertex s. These new vertices, which are adjacent to level 1 vertices and not previously assigned to a level, are placed into level 2, and so on. The BFS traversal terminates when every vertex has been visited.



In [13]:
class BreadthFirst(Search):
    def search(self):
        start = self.search_grid.grid[self.search_grid.start_pos[0], \
            self.search_grid.start_pos[1]]
        goal = self.search_grid.grid[self.search_grid.goal_pos[0], \
            self.search_grid.goal_pos[1]]
        self.fringe.append(start)         

        while True:
            if len(self.fringe) == 0:
                return False
            current = self.fringe.pop(0)
            if self.visited.count(current) == 0:
                self.expanded.append(current) 
                self.visited.append(current)  
            else:
                continue                
            if current == goal:
                self.set_path(goal)
                return True
            
            self.__expand(current)          
        
    def __expand(self, current):

        pos = self.search_grid.position(current)    
        current_fringe = []
        if pos[0] > 0:
            current_fringe.append(self.search_grid.grid[pos[0] - 1, pos[1]])
        if pos[1] > 0:
            current_fringe.append(self.search_grid.grid[pos[0], pos[1] - 1])
        if pos[1] < (self.search_grid.grid.shape[1] - 1):
            current_fringe.append(self.search_grid.grid[pos[0], pos[1] + 1])
        if pos[0] < (self.search_grid.grid.shape[0] - 1):
            current_fringe.append(self.search_grid.grid[pos[0] + 1, pos[1]])
        for value in current_fringe:
            if value != -1:
                self.fringe.append(value)
                self.node_parent_list.append((value, current)) 

In [39]:
sol1 = BreadthFirst(grid)
sol1.search()
print "\nExpanded: ", sol1.expanded
print "Path: ", sol1.path
print "Path Cost: ",


if (len(sol1.path) > 0):
    print len(sol1.path) - 1
else:
    print "Failure"


Expanded:  [44, 36, 45, 52, 35, 37, 46, 53, 51, 60, 38, 54, 61, 50, 59, 39, 55, 62, 42, 49, 58, 40, 56, 63, 41, 57, 32, 48, 64, 33, 24, 25, 16, 23, 17, 26, 8, 15, 22, 9, 18, 7, 14, 21, 1, 10, 19]
Path:  [44, 52, 51, 50, 49, 41, 33, 25, 26, 18, 19]
Path Cost:  10


### 4. A-star algorithm

$A^*$ is an informed search algorithm, or a best-first search, meaning that it solves problems by searching among all possible paths to the solution (goal) for the one that incurs the smallest cost (least distance travelled, shortest time, etc.), and among these paths it first considers the ones that appear to lead most quickly to the solution. It is formulated in terms of weighted graphs: starting from a specific node of a graph, it constructs a tree of paths starting from that node, expanding paths one step at a time, until one of its paths ends at the predetermined goal node.

At each iteration of its main loop, $A^*$ needs to determine which of its partial paths to expand into one or more longer paths. It does so based on an estimate of the cost (total weight) still to go to the goal node. Specifically, $A^*$ selects the path that minimizes
$$f(n) = g(n) + h(n) $$
    
where $n$ is the last node on the path, $g(n)$ is the cost of the path from the start node to $n$, and $h(n)$ is a heuristic that estimates the cost of the cheapest path from $n$ to the goal. The heuristic is problem-specific. For the algorithm to find the actual shortest path, the heuristic function must be admissible, meaning that it never overestimates the actual cost to get to the nearest goal node.

In [40]:
class AStar(Search):
    def search(self):
        
        start = self.search_grid.grid[self.search_grid.start_pos[0], \
            self.search_grid.start_pos[1]]
        goal = self.search_grid.grid[self.search_grid.goal_pos[0], \
            self.search_grid.goal_pos[1]]
        self.fringe.append((start, self.search_grid.heuristic(start)))

        while True:
            if len(self.fringe) == 0:
                return False
            lowest_index = self.__least_cost_node_index(self.fringe)
            current = self.fringe.pop(lowest_index)
            
            if self.visited.count(current[0]) == 0:
                self.expanded.append(current)   
                self.visited.append(current[0])     
            else:
                continue            
            
            if current[0] == goal:
                self.set_path(goal)
                return True
            
            self.__expand(current[0])   
        
    def __least_cost_node_index(self, fringe):
        lowest_value = np.float("inf")
        lowest_index = 0
        current_position = 0

        for node in fringe:
            if node[1] <= lowest_value:
                lowest_value = node[1]
                lowest_index = current_position
            current_position += 1
            
        return lowest_index

    def __expand(self, current):
       

        pos = self.search_grid.position(current)    
        current_fringe = []
        path_cost = 1
        if len(self.node_parent_list) != 0:
            path_cost = self.__path_cost(current)
        if pos[0] > 0:
            current_fringe.append(self.search_grid.grid[pos[0] - 1, pos[1]])
        if pos[1] > 0:
            current_fringe.append(self.search_grid.grid[pos[0], pos[1] - 1])
        if pos[1] < (self.search_grid.grid.shape[1] - 1):
            current_fringe.append(self.search_grid.grid[pos[0], pos[1] + 1])
        if pos[0] < (self.search_grid.grid.shape[0] - 1):
            current_fringe.append(self.search_grid.grid[pos[0] + 1, pos[1]])

        for value in current_fringe:
            if value != -1:
                total_cost = path_cost + self.search_grid.heuristic(value)
                self.fringe.append((value, total_cost))
                self.node_parent_list.append((value, current, path_cost))    

    def __path_cost(self, value):
        for node in self.node_parent_list:
            if node[0] == value:
                return node[2] + 1


In [41]:
sol2 = AStar(grid)
sol2.search()
print "\nExpanded: ", sol2.expanded
print "Path: ", sol2.path
print "Path Cost: ",
if (len(sol2.path) > 0):
    print len(sol2.path) - 1
else:
    print "Failure"


Expanded:  [(44, 4), (36, 4), (35, 4), (37, 6), (52, 6), (51, 6), (45, 6), (53, 8), (46, 8), (38, 8), (59, 8), (50, 8), (42, 8), (60, 8), (61, 10), (41, 10), (33, 10), (25, 10), (26, 10), (18, 10), (19, 10)]
Path:  [44, 52, 51, 50, 42, 41, 33, 25, 26, 18, 19]
Path Cost:  10


### Conclusion

**BFS** works by expanding the start node and adding its successors into the $\texttt{fringe}$, and then the successors of the start node are similarly expanded, and so on. In other words, all nodes at a given depth in the search tree are expanded before moving on to the next. This process is repeated until either the goal is found or all possible nodes have been expanded.

BFS is complete in the sense that the algorithm will always find the goal if it exists. The path from start to goal is also always optimal.

However, since BFS probes through the entire search space at each level in the search tree, it has very large time and space-complexity. The problem worsens  if the goal node is many steps away from the start. Therefore, BFS is not a suitable strategy when a large search space is involved.

**DFS** works a little bit differently than BFS. It always expands the deepest node in the $\texttt{fringe}$ of the search tree. This process is repeated until either the goal is found or all possible nodes have been expanded.

Like BFS, DFS is also complete, but only because the depth is bounded by the size of the search grid. However, the path found is not always optimal depending on where the goal is located on the search grid.  In general, it is accepted that DFS is not optimal.

DFS has a low space-complexity since it only expands one node at a time on each level on the search tree. When that node's descendants have been fully explored, it can be safely deleted from memory. Its time-complexity may differ, however, again depending on where the goal is located on search grid.

**$A^*$** search is both complete and optimal only if the heuristic $h(x)$ is admissible. Admissible heuristics are optimistic -- the value is always less than or equal to the true cost. In this case, $h(x)$ is derived from a relaxed version of the grid, i.e. grids with no forbidden nodes. The $h(x)$ value is then done by simply calculating the differences between the coordinates of the node and the goal node. This formula proves to be admissible because it never overestimates the cost to goal node and is actually equal to the true cost in a relaxed grid.

**$A^*$ ** search has very high time and space-complexity. $A^*$ stores all nodes in memory. Its time-complexity is greatly affected by the relative error in $h(x)$ and by the length of solution. However, in a small workspace with good $h(x)$ and $g(x)$ evaluation fuctions, $A^*$ search works very well.

